In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py --style=tree --animation=spacetime

In [ ]:
def setInstance(instance):
    global i
    global f
    global o_verify
    
    global S
    global W
    global Q
    
    if instance == "sparse":
        i = Tensor.fromUncompressed(["W"], [1, 0, 2, 0, 3, 1, 2, 3])
        f = Tensor.fromUncompressed(["S"], [1, 0, 3])
        o_verify = Tensor.fromUncompressed(["Q"], [7, 0, 11, 3, 9, 10])
    else:
        i = Tensor.fromUncompressed(["W"], [1, 4, 2, 4, 3, 1, 2, 3])
        f = Tensor.fromUncompressed(["S"], [1, 2, 3])
        o_verify = Tensor.fromUncompressed(["Q"], [15, 20, 19, 13, 11, 14])

    i.setName("I").setColor("blue")
    f.setName("F").setColor("green")
    
    S = f.getShape()[0]
    W = i.getShape()[0]
    Q = W-S+1

    print(f"W = {W}")
    print(f"S = {S}")
    print(f"Q = {Q}")
    print("")
    
    print("Input activations")
    displayTensor(i)
    print("Filter Weights")
    displayTensor(f)
    print("Output activations (expected)")
    displayTensor(o_verify)


select_instance = interactive(setInstance,
                              instance=["dense", "sparse"])

createRunallButton()
display(select_instance)



In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

f_s = f.getRoot()
i_w = i.getRoot()
o_q = o.getRoot()

print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

for q, (o_ref, _) in o_q << output_shape:
    print(f"Processing output: ({q}, ({o_ref}))")
    for pe, (w, (f_val, i_val)) in enumerate(f_s.project(lambda s: q+s) & i_w):
        print(f"  Processing weights and activations ({w}, ({f_val}, {i_val})")
        o_ref += f_val * i_val
        addActivity(canvas, (w-q,), (w,), (q,), worker=f"PE{pe}")
    addFrame(canvas)

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

f_s = f.getRoot()
i_w = i.getRoot()
o_q = o.getRoot()

print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

for q, (o_ref, _) in o_q << output_shape:
    print(f"Processing output: ({q}, ({o_ref}))")
    for pe, (w, (f_val, i_val)) in enumerate(f_s.project(lambda s: q+s) & i_w):
        print(f"  Processing weights and activations ({w}, ({f_val}, {i_val})")
        o_ref += f_val * i_val
        addActivity(canvas, (w-q,), (w,), (q,), worker=f"PE{pe}", skew=pe)
    addFrame(canvas)

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify